In [51]:
#package
using ConcurrentSim
using ResumableFunctions
using Random
using StableRNGs
using Distributions
using StatsBase
using Statistics

In [52]:
const seed1 = 111
const seed2 = 222
const seed3 = 333
const seed4 = 444
const seed5 = 555
const seed6 = 666
const seed7 = 777
const seed8 = 888

888

In [53]:
#Kedatangan pelanggan (interval waktu antara satu pelanggan tiba dan pelanggan berikutnya)
arrival_rng = StableRNG(seed1)
function arrival1() #sesi 1
    return 88*randexp(arrival_rng, Float64)  #distribusi eksponensial dgn rata2 93 detik
end

function arrival2() #sesi 2
    return 72*randexp(arrival_rng, Float64)  #distribusi eksponensial dgn rata2 93 detik
end

function arrival3() #sesi 3
    return 105*randexp(arrival_rng, Float64)  #distribusi eksponensial dgn rata2 93 detik
end
function arrival4() #sesi 4
    return 123*randexp(arrival_rng, Float64)  #distribusi eksponensial dgn rata2 93 detik
end

#Lama pelanggan berada di dine in
leaving_rng = StableRNG(seed2)
function leaving()
    return 2839*randexp(leaving_rng, Float64) #distribusi eksponensial dgn rata2 2839 detik
end

# Lama proses pelayanan di loket Pemesanan (pengemasan makanan)
ordering1_rng = StableRNG(seed3)
function ordering1()
    return 39*randexp(ordering1_rng, Float64)
end

#Lama proses pelayanan di Loket Pembayaran (include pemesanan minuman dan merch)
ordering2_rng = StableRNG(seed4)
function ordering2()
    return 91*randexp(ordering2_rng, Float64)
end

#Lama proses pelayanan di Loket Snackbox
ordering0_rng = StableRNG(1010)
function ordering0()
    return 78.33*randexp(ordering2_rng, Float64)
end

# Probabilitas pemilihan loket (Menentukan ke mana pelanggan akan pergi, apakah ke loket 1 atau loket 2)
loket_rng = StableRNG(seed5)
function cust_dest()
    dest = [1, 2, 0]  # Pilihan hanya antara 1 dan 2, dan 0 untuk loket snackbox
    dest_weights = [141/155, 3/155, 9/155]
    return sample(loket_rng, dest, Weights(dest_weights))
end
    
# Probabilitas ukuran pelanggan
size_rng = StableRNG(seed6)
function cust_size()
    size = 1:5  #isinya 1-5 orang tiap grup
    size_weights = [0.755, 0.219, 0.019, 0, 0.006] #50% utk 1 orang, 30% utk 2 org, 10% utk 3/4 org
    return sample(size_rng, size, Weights(size_weights))
end
        
#Probabilitas dine in atau take away
duduk_rng = StableRNG(seed7)
function stat_duduk()
    choice = ["Dine In", "Take Away"] #3 untuk dinein, 7 untuk takeaway
    choice_weights = [0.24, 0.76] #30% dine in, 70% takeaway
    return sample(duduk_rng, choice, Weights(choice_weights))
end
            
#Probabilitas pelanggan mesen lagi apapun itu
again_rng = StableRNG(seed8)
function order_again()
    again = ["ya", "tidak"]
    again_weights = [0.095, 0.905]
    return sample(again_rng, again, Weights(again_weights))
end

order_again (generic function with 1 method)

In [54]:
mutable struct CUSTOMER
    is_balking::Bool
    is_dineIn::String
    next_order::String
    pilih_loket::Int
    #id::Int
    #group_id::Int
    function CUSTOMER()
        cust = new()
        cust.is_balking = false
        cust.is_dineIn = "Take Away"
        cust.next_order = "tidak"
        cust.pilih_loket = 1 #nilai awal
        return cust
    end
end

mutable struct BAKERY
    LOKET1::Resource
    LOKET2::Resource
    LOKET0::Resource
    LOKET1_queue_count::Int
    LOKET2_queue_count::Int
    LOKET0_queue_count::Int
    KURSI::Resource 
    KURSI_count::Int #untuk melacak kursi tersedia
    areaUnderQ1::Float64 #luas daerah Q untuk menghitung delay time untuk loket 1
    areaUnderQ2::Float64 #luas daerah Q untuk menghitung delay time untuk loket 2
    areaUnderQ0::Float64 
    last_event_time1::Float64 #waktu perubahan terakhir perubahan antrean loket 1
    last_event_time2::Float64 #waktu perubahan terakhir perubahan antrean loket 2
    last_event_time0::Float64
    balk_count1::Int #Jumlah pelanggan balk di loket 1
    balk_count2::Int #Jumlah pelanggan balk di loket 2
    balk_count0::Int
    dinein_rejected_count::Int #Jumlah pelanggan yang ingin dine in tp kursi penuh
    
    function BAKERY(env::Simulation)
        bakery = new()
        # Loket pemesanan dan pembayaran masing-masing hanya melayani 1 pelanggan pada satu waktu
        bakery.LOKET1 = Resource(env, 1)
        bakery.LOKET1_queue_count = 0 #inisialisasi pembatasan resource antrian loket 1
        bakery.LOKET2 = Resource(env, 1)
        bakery.LOKET2_queue_count = 0 #inisialisasi pembatasan resource antrian loket 2
        bakery.LOKET0 = Resource(env, 1)
        bakery.LOKET0_queue_count = 0
        # Tempat duduk tersedia untuk maksimal 60 pelanggan
        bakery.KURSI = Resource(env, 45)
        bakery.KURSI_count = 45 #inisialisasi jumlah kursi
        bakery.areaUnderQ1 = 0.0 #inisialisasi area di bawah grafik menghitung panjang antrian thd waktiu
        bakery.areaUnderQ2 = 0.0 #buat loket 2
        bakery.areaUnderQ0 = 0.0
        bakery.last_event_time1 = 0.0
        bakery.last_event_time2 = 0.0
        bakery.last_event_time0 = 0.0
        bakery.balk_count1 = 0
        bakery.balk_count2 = 0
        bakery.balk_count0 = 0
        bakery.dinein_rejected_count = 0
        return bakery
    end
end

mutable struct CATATAN
    wait_loket1
    wait_loket2 
    wait_loket0
    function CATATAN()
        ct = new()
        ct.wait_loket1 = []
        ct.wait_loket2 = []
        ct.wait_loket0 = []
        return ct
    end
end

In [55]:
#Fungsi pembangkit proses kedatangan pelanggan
@resumable function arrival_customer(env::Simulation, bakery::BAKERY)
    grup_id = 1
    while true
        sim_time = now(env)
        if sim_time <=3600
            @yield timeout(env, arrival1()) #menunggu selama waktu tertentu hingga pelanggan berikutnya dtg
        elseif sim_time >3600 && sim_time <=7200
            @yield timeout(env, arrival2())
        elseif sim_time > 7200 && sim_time <= 10800
            @yield timeout(env, arrival3())
        elseif sim_time > 10800 && sim_time <=14400
            @yield timeout(env, arrival4())
        end
        grup_size = cust_size() #paukai fungsi probabilitas ukuran pelanggan di awal
        
        #memanggil cust_process utk setiap pelanggan pergrupnya
        @process cust_process(env, grup_id, grup_size, cust, bakery) #parameter2 proses setiap pelanggannya dlm melakukan proses simulasi scr paralel
        grup_id +=1
    end
end

arrival_customer (generic function with 1 method)

In [56]:
global total_customers = 0  # Variabel untuk melacak total jumlah pelanggan

0

In [57]:
#Fungsi pemroses kegiatan pelanggan
@resumable function cust_process(env::Simulation, grup_id, grup_size, cust::CUSTOMER, bakery::BAKERY)
    global total_customers  # Mengakses variabel global
    total_customers += grup_size  # Tambahkan jumlah pelanggan dalam grup ke total
    println("Total pelanggan sejauh ini: $total_customers")
    
    
    println("\e[1mPELANGGAN DATANG!\e[0m")
    println("Group ID: $grup_id")  #id pelanggan
    println("Waktu Kedatangan: $(round(now(env),digits=2))") #waktu kedatangan pembulatan 2 digit
    println("Jumlah Pelanggan: $grup_size") #jumlah pelanggan dalam kelompoknya
    println("      \e[1m***\e[0m         ")
    
    cust.pilih_loket = cust_dest() #pakai fungsi probabilitas destinasi loket pelanggan di awal
        
    #Jika pelanggan memilih loket 1 Pemesanan Makanan terlebih dahulu :
    if cust.pilih_loket == 1
    #cek apakah antrian loket 1 penuh (kapasitas 12)
        if bakery.LOKET1_queue_count >= 12
            println("Jumlah pelanggan mengantre di Loket 1: $(bakery.LOKET1_queue_count). Pelanggan dari Kelompok $grup_id balk dari loket 1, antrian penuh.")
            bakery.balk_count1 +=1 #tambahkan pelanggan balk loket 1 ke statistik
            cust.is_balking = true #tandai sebagai pelanggan yg balk
            return #keluar dari sistem/bakery
        end
            
        #jika antrian belum penuh, lanjutkan masuk antrian loket 1
        start1 = round(now(env),digits=2) #cetak waktu skrg waktu kedatangan untuk waktu mulai antre
        println("Pelanggan dari Kelompok $grup_id memasuki antrean loket 1 pada $(round(start1,digits=2))")
        
        # Ketika pelanggan masuk antrean (untuk areaUnderQ)
        duration = round(now(env), digits=2) - bakery.last_event_time1
        bakery.areaUnderQ1 += duration * bakery.LOKET1_queue_count
        bakery.last_event_time1 = round(now(env), digits=2)
        bakery.LOKET1_queue_count +=1 #tambah jumlah antrian loket 1
        println("Jumlah pelanggan di Loket 1: $(bakery.LOKET1_queue_count)")
        
        @yield request(bakery.LOKET1) #merequest loket 1, antre kalo server sedang melayani
        
        wait1 = round(now(env)- start1,digits=2) #cetak waktu tunggu
        push!(ct.wait_loket1, wait1) #push ke vector wait_loket1 di awal
        #bakery.areaUnderQ1 += (round(now(env)- start2,digits=2))*bakery.LOKET1_queue_count #hitung area bawah grafik panjang antrian thdp waktu
        #durasi berada di antrian dikali panjang antrian, kemudian ditambahkan ke areaUnderQ
        
        pesan_loket1 = @process bakery_process(env, grup_id, grup_size, cust, bakery)
        @yield pesan_loket1 #proses pemesanan di loket 1
        @yield release(bakery.LOKET1) #melepaskan loket 1 setelah selesai pelayanan
        
        # Ketika pelanggan keluar antrean (untuk areaUnderQ)
        duration = round(now(env), digits=2) - bakery.last_event_time1
        bakery.areaUnderQ1 += duration * bakery.LOKET1_queue_count
        bakery.last_event_time1 = round(now(env), digits=2)    
        bakery.LOKET1_queue_count -=1 #kurangi jumlah antrian loket 1 setelah selesai pelayanan loket 1
        println("Pelanggan dari Kelompok $grup_id selesai di loket 1 menuju antrian loket 2 pada $(round(now(env),digits=2))") 
        println("Jumlah pelanggan di Loket 1: $(bakery.LOKET1_queue_count)")
        
        #setelah Loket 1 pelanggan langsung masuk antrian loket 2
        start2 = round(now(env),digits=2) #cetak waktu skrg waktu kedatangan untuk waktu mulai antri loket 2
        println("Pelanggan dari Kelompok $grup_id memasuki antrean loket 2 pada $(round(start2, digits=2))")
        
        # Ketika pelanggan masuk antrean (untuk areaUnderQ)
        duration = round(now(env), digits=2) - bakery.last_event_time2
        bakery.areaUnderQ2 += duration * bakery.LOKET2_queue_count
        bakery.last_event_time2 = round(now(env), digits=2)
        bakery.LOKET2_queue_count +=1 #tambah jumlah antrian loket 2
        println("Jumlah pelanggan di Loket 2: $(bakery.LOKET2_queue_count)")
        
        @yield request(bakery.LOKET2)
    
        wait2 = round(now(env)- start2,digits=2) #cetak waktu tunggu
        push!(ct.wait_loket2, wait2) #push ke vector wait_loket2 di awal
        #bakery.areaUnderQ2 += (round(now(env)- start2,digits=2))*bakery.LOKET2_queue_count #hitung area bawah grafik panjang antrian thdp waktu
        #(durasi berada di antrian dikali panjang antrian, kemudian ditambahkan ke areaUnderQ)
        pesan_loket2 = @process bakery_process(env, grup_id, grup_size, cust, bakery)
        @yield pesan_loket2 #proses pemesanan dan pembayaran di loket 2
        
        @yield release(bakery.LOKET2) #melepaskan loket 2 setelah selesai pelayanan
        
         # Ketika pelanggan keluar antrean (untuk areaUnderQ)
        duration = round(now(env), digits=2) - bakery.last_event_time2
        bakery.areaUnderQ2 += duration * bakery.LOKET2_queue_count
        bakery.last_event_time2 = round(now(env), digits=2)
        bakery.LOKET2_queue_count -=1 #kurangi jumlah antrian loket 1 setelah selesai pelayanan loket
        println("Pelanggan dari Kelompok $grup_id selesai di loket 2 pada $(round(now(env),digits=2))")
        println("Jumlah pelanggan di Loket 2: $(bakery.LOKET2_queue_count)")
        
    #Jika pelanggan memilih loket 2 langsung Pemesanan Minuman/Merch dan melakukan pembayaran
    elseif cust.pilih_loket == 2
        #cek apakah antrian loket 1 penuh (kapasitas 12)
        if bakery.LOKET2_queue_count >= 12
            println("Jumlah pelanggan di Loket 2: $(bakery.LOKET2_queue_count)")
            println("Pelanggan dari Kelompok $grup_id balk dari loket 2, antrian penuh.")
            bakery.balk_count2 +=1 #tambahkan pelanggan balk loket 2 ke statistik
            cust.is_balking = true #tandai sebagai pelanggan yg balk
            return #keluar dari sistem/bakery
        end
        #jika antrian belum penuh, lanjutkan masuk antrian loket 2
        start2 = round(now(env),digits=2)
        println("Pelanggan dari Kelompok $grup_id LANGSUNG memasuki antrian loket 2 pada $(round(start2,digits=2))")
        
        # Ketika pelanggan masuk antrean (untuk areaUnderQ)
        duration = round(now(env), digits=2) - bakery.last_event_time2
        bakery.areaUnderQ2 += duration * bakery.LOKET2_queue_count
        bakery.last_event_time2 = round(now(env), digits=2)
        bakery.LOKET2_queue_count +=1 #tambah jumlah antrian loket 2
        println("Jumlah pelanggan di Loket 2: $(bakery.LOKET2_queue_count)")
        
        @yield request(bakery.LOKET2) #merequest loket 2, antri kalo server penuh
        
        wait2 = round(now(env)- start2,digits=2) #cetak waktu tunggu 
        push!(ct.wait_loket2, wait2)
        #bakery.areaUnderQ2 += (round(now(env)- start2,digits=2))*bakery.LOKET2_queue_count #hitung area bawah grafik panjang antrian thdp waktu
        #(durasi berada di antrian dikali panjang antrian, kemudian ditambahkan ke areaUnderQ)
        
        pesan_loket2 = @process bakery_process(env, grup_id, grup_size, cust, bakery)
        @yield pesan_loket2 #proses pemesanan dan pembayaran di loket 2
        @yield release(bakery.LOKET2) #melepaskan loket 2 setelah selesai pelayanan
        
         # Ketika pelanggan keluar antrean (untuk areaUnderQ)
        duration = round(now(env), digits=2) - bakery.last_event_time2
        bakery.areaUnderQ2 += duration * bakery.LOKET2_queue_count
        bakery.last_event_time2 = round(now(env), digits=2)
        bakery.LOKET2_queue_count -=1 #kurangi jumlah antrian loket 2 setelah selesai pelayanan loket
        println("Pelanggan dari Kelompok $grup_id selesai di loket 2 pada $(round(now(env),digits=2))")
        println("Jumlah pelanggan di Loket 2: $(bakery.LOKET2_queue_count)")
        
    #Jika pelanggan memilih loket Snackbox terlebih dahulu :
    elseif cust.pilih_loket == 0
    #cek apakah antrian loket S penuh (kapasitas 12)
        if bakery.LOKET0_queue_count >= 12
            println("Jumlah pelanggan mengantre di Loket Snackbox: $(bakery.LOKET0_queue_count). Pelanggan dari Kelompok $grup_id balk dari loket Snackbox, antrian penuh.")
            bakery.balk_count0 +=1 #tambahkan pelanggan balk loket 1 ke statistik
            cust.is_balking = true #tandai sebagai pelanggan yg balk
            return #keluar dari sistem/bakery
        end
            
        #jika antrian belum penuh, lanjutkan masuk antrian loket 1
        start0 = round(now(env),digits=2) #cetak waktu skrg waktu kedatangan untuk waktu mulai antre
        println("Pelanggan dari Kelompok $grup_id memasuki antrean loket Snackbox pada $(round(start0,digits=2))")
        
        # Ketika pelanggan masuk antrean (untuk areaUnderQ)
        duration = round(now(env), digits=2) - bakery.last_event_time0
        bakery.areaUnderQ0 += duration * bakery.LOKET0_queue_count
        bakery.last_event_time0 = round(now(env), digits=2)
        bakery.LOKET0_queue_count +=1 #tambah jumlah antrian loket 1
        println("Jumlah pelanggan di Loket Snackbox: $(bakery.LOKET0_queue_count)")
        
        @yield request(bakery.LOKET0) #merequest loket Snackbox, antre kalo server sedang melayani
        
        wait0 = round(now(env)- start0,digits=2) #cetak waktu tunggu
        push!(ct.wait_loket0, wait0) #push ke vector wait_loket0 di awal
        #bakery.areaUnderQ0 += (round(now(env)- start0,digits=2))*bakery.LOKET0_queue_count #hitung area bawah grafik panjang antrian thdp waktu
        #durasi berada di antrian dikali panjang antrian, kemudian ditambahkan ke areaUnderQ
        
        pesan_loket0 = @process bakery_process(env, grup_id, grup_size, cust, bakery)
        @yield pesan_loket0 #proses pemesanan di loket S
        @yield release(bakery.LOKET0) #melepaskan loket S setelah selesai pelayanan
        
        # Ketika pelanggan keluar antrean (untuk areaUnderQ)
        duration = round(now(env), digits=2) - bakery.last_event_time0
        bakery.areaUnderQ0 += duration * bakery.LOKET0_queue_count
        bakery.last_event_time0 = round(now(env), digits=2)    
        bakery.LOKET0_queue_count -=1 #kurangi jumlah antrian loket S setelah selesai pelayanan loket 1
        println("Pelanggan dari Kelompok $grup_id selesai di loket Snackbox menuju antrian loket 2 pada $(round(now(env),digits=2))") 
        println("Jumlah pelanggan di Loket Snackbox: $(bakery.LOKET1_queue_count)")
        
        #setelah Loket 1 pelanggan langsung masuk antrian loket 2
        start2 = round(now(env),digits=2) #cetak waktu skrg waktu kedatangan untuk waktu mulai antri loket 2
        println("Pelanggan dari Kelompok $grup_id memasuki antrean loket 2 pada $(round(start2, digits=2))")
        
        # Ketika pelanggan masuk antrean (untuk areaUnderQ)
        duration = round(now(env), digits=2) - bakery.last_event_time2
        bakery.areaUnderQ2 += duration * bakery.LOKET2_queue_count
        bakery.last_event_time2 = round(now(env), digits=2)
        bakery.LOKET2_queue_count +=1 #tambah jumlah antrian loket 2
        println("Jumlah pelanggan di Loket 2: $(bakery.LOKET2_queue_count)")
        
        @yield request(bakery.LOKET2)
    
        wait2 = round(now(env)- start2,digits=2) #cetak waktu tunggu
        push!(ct.wait_loket2, wait2) #push ke vector wait_loket2 di awal
        #bakery.areaUnderQ2 += (round(now(env)- start2,digits=2))*bakery.LOKET2_queue_count #hitung area bawah grafik panjang antrian thdp waktu
        #(durasi berada di antrian dikali panjang antrian, kemudian ditambahkan ke areaUnderQ)
        pesan_loket2 = @process bakery_process(env, grup_id, grup_size, cust, bakery)
        @yield pesan_loket2 #proses pemesanan dan pembayaran di loket 2
        
        @yield release(bakery.LOKET2) #melepaskan loket 2 setelah selesai pelayanan
        
         # Ketika pelanggan keluar antrean (untuk areaUnderQ)
        duration = round(now(env), digits=2) - bakery.last_event_time2
        bakery.areaUnderQ2 += duration * bakery.LOKET2_queue_count
        bakery.last_event_time2 = round(now(env), digits=2)
        bakery.LOKET2_queue_count -=1 #kurangi jumlah antrian loket 1 setelah selesai pelayanan loket
        println("Pelanggan dari Kelompok $grup_id selesai di loket 2 pada $(round(now(env),digits=2))")
        println("Jumlah pelanggan di Loket 2: $(bakery.LOKET2_queue_count)")
            
    end
        
    #menentukan dine in atau take away
    cust.is_dineIn = stat_duduk() #jika pelanggan ingin duduk, statusnya dine in
    if cust.is_dineIn == "Dine In"
        println("Pelanggan dari Kelompok $grup_id memilih Dine-in")
        
        #cek ketersediaan kursi sebelum merequest kursi
        if bakery.KURSI_count >= grup_size #apakah ada kursi yang belum terpakai sejumlah grup
            @yield request(bakery.KURSI) #merequest kursi tersedia
            bakery.KURSI_count -= grup_size #kurangi jumlah kursi yang tersedia dengan jumlah grup             println("Pelanggan dari Kelompok $grup_id menempati kursi pada $(round(now(env),digits=2))")
            start_time = round(now(env), digits=2) #waktu mulai pelanggan dine in duduk
            println("Pelanggan dari Kelompok $grup_id menempati $grup_size kursi pada $(round(now(env),digits=2))")
            println("Kursi tersisa : $(bakery.KURSI_count)")  
            
            @yield timeout(env, leaving()) #pelanggan dine in sampai waktu dr fungsi leaving() di awal
            @yield release(bakery.KURSI) #melepaskan kursi setelah selesai proses dine in
            println("Pelanggan dari Kelompok $grup_id selesai Dine-In pada $(round(now(env),digits=2))")
            bakery.KURSI_count += grup_size #tambahkan jumlah kursi yang tersedia dengan 1
            println("Kursi bertambah. Jumlah kursi yang tersedia : $(bakery.KURSI_count)")
                
            #Apakah pelanggan yang dine in mau memesan lagi?
            #di tengah dine in, untuk pelanggan yang ingin mesan lagi
            cust.next_order = order_again() #pakai probabilitas pesan lagi di awal
            if cust.next_order == "ya"
                println("Pelanggan dari Kelompok $grup_id memutuskan memesan lagi pada $(round(now(env),digits=2))")
                cust_process(env, grup_id, grup_size, cust, bakery)  # Panggil kembali fungsi untuk pesanan berikutnya
            end
        else
            println("Pelanggan dari Kelompok $grup_id ingin Dine-In, tetapi kursi penuh. Beralih ke Take-Away")
            bakery.dinein_rejected_count += grup_size
        end
    else
        
        println("Pelanggan dari Kelompok $grup_id memilih Take-Away")
        cust.next_order == "tidak"
    end
end

cust_process (generic function with 1 method)

In [58]:
#Fungsi pemroses kejadian di server
@resumable function bakery_process(env::Simulation, grup_id, grup_size, cust::CUSTOMER, bakery::BAKERY)
    if cust.pilih_loket == 1
        println("Loket Pemesanan menerima pelanggan dari kelompok $grup_id pada $(round(now(env),digits=2))")
        @yield timeout(env,ordering1()) #sampai waktunya pk fungsi lama pelayanan loket 1 di awal
        println("Pelanggan dari kelompok $grup_id meninggalkan Loket Pemesanan")
    elseif cust.pilih_loket == 2
        println("Loket Pembayaran menerima pelanggan dari kelompok $grup_id pada $(round(now(env),digits=2))")
        @yield timeout(env, ordering2()) #sampai waktunya pk fungsi lama pelayanan loket 2 di awal
        println("Pelanggan dari kelompok $grup_id meninggalkan Loket Pembayaran")
    elseif cust.pilih_loket == 0
        println("Loket Snackbox menerima pelanggan dari kelompok $grup_id pada $(round(now(env),digits=2))")
        @yield timeout(env, ordering0()) #sampai waktunya pk fungsi lama pelayanan loket S di awal
        println("Pelanggan dari kelompok $grup_id meninggalkan Loket Snackbox")    
    end
end

bakery_process (generic function with 1 method)

In [59]:
env = Simulation()
cust = CUSTOMER() #inisialisasi ulang CUSTOMER pada setiap iterasi
ct = CATATAN()
bakery = BAKERY(env)
@process arrival_customer(env, bakery)

run(env,14400)

Total pelanggan sejauh ini: 1
PELANGGAN DATANG!
Group ID: 1
Waktu Kedatangan: 82.62
Jumlah Pelanggan: 1
      ***         
Pelanggan dari Kelompok 1 memasuki antrean loket 1 pada 82.62
Jumlah pelanggan di Loket 1: 1
Loket Pemesanan menerima pelanggan dari kelompok 1 pada 82.62
Pelanggan dari kelompok 1 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 1 selesai di loket 1 menuju antrian loket 2 pada 95.39
Jumlah pelanggan di Loket 1: 0
Pelanggan dari Kelompok 1 memasuki antrean loket 2 pada 95.39
Jumlah pelanggan di Loket 2: 1
Loket Pemesanan menerima pelanggan dari kelompok 1 pada 95.39
Pelanggan dari kelompok 1 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 1 selesai di loket 2 pada 115.82
Jumlah pelanggan di Loket 2: 0
Pelanggan dari Kelompok 1 memilih Take-Away
Total pelanggan sejauh ini: 2
PELANGGAN DATANG!
Group ID: 2
Waktu Kedatangan: 255.63
Jumlah Pelanggan: 1
      ***         
Pelanggan dari Kelompok 2 memasuki antrean loket 1 pada 255.63
Jumlah pelanggan di Loket 1:

Pelanggan dari Kelompok 13 memasuki antrean loket 2 pada 1098.47
Jumlah pelanggan di Loket 2: 2
Total pelanggan sejauh ini: 17
PELANGGAN DATANG!
Group ID: 14
Waktu Kedatangan: 1115.53
Jumlah Pelanggan: 1
      ***         
Pelanggan dari Kelompok 14 memasuki antrean loket 1 pada 1115.53
Jumlah pelanggan di Loket 1: 1
Loket Pemesanan menerima pelanggan dari kelompok 14 pada 1115.53
Pelanggan dari kelompok 12 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 12 selesai di loket 2 pada 1133.13
Jumlah pelanggan di Loket 2: 1
Pelanggan dari Kelompok 12 memilih Dine-in
Pelanggan dari Kelompok 12 menempati 1 kursi pada 1133.13
Kursi tersisa : 36
Loket Pemesanan menerima pelanggan dari kelompok 13 pada 1133.13
Pelanggan dari kelompok 13 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 13 selesai di loket 2 pada 1145.61
Jumlah pelanggan di Loket 2: 0
Pelanggan dari Kelompok 13 memilih Take-Away
Pelanggan dari kelompok 14 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 14 selesai di 

Loket Pemesanan menerima pelanggan dari kelompok 25 pada 2094.57
Pelanggan dari kelompok 25 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 25 selesai di loket 1 menuju antrian loket 2 pada 2114.45
Jumlah pelanggan di Loket 1: 0
Pelanggan dari Kelompok 25 memasuki antrean loket 2 pada 2114.45
Jumlah pelanggan di Loket 2: 2
Pelanggan dari kelompok 24 meninggalkan Loket Snackbox
Pelanggan dari Kelompok 24 selesai di loket 2 pada 2195.99
Jumlah pelanggan di Loket 2: 1
Pelanggan dari Kelompok 24 memilih Dine-in
Pelanggan dari Kelompok 24 menempati 2 kursi pada 2195.99
Kursi tersisa : 29
Loket Pemesanan menerima pelanggan dari kelompok 25 pada 2195.99
Pelanggan dari Kelompok 11 selesai Dine-In pada 2256.1
Kursi bertambah. Jumlah kursi yang tersedia : 30
Total pelanggan sejauh ini: 34
PELANGGAN DATANG!
Group ID: 26
Waktu Kedatangan: 2258.78
Jumlah Pelanggan: 3
      ***         
Pelanggan dari Kelompok 26 memasuki antrean loket 1 pada 2258.78
Jumlah pelanggan di Loket 1: 1
Loket Pemesan

Pelanggan dari Kelompok 38 memasuki antrean loket 1 pada 2911.04
Jumlah pelanggan di Loket 1: 3
Pelanggan dari kelompok 35 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 35 selesai di loket 2 pada 2912.93
Jumlah pelanggan di Loket 2: 0
Pelanggan dari Kelompok 35 memilih Take-Away
Pelanggan dari Kelompok 22 selesai Dine-In pada 2914.61
Kursi bertambah. Jumlah kursi yang tersedia : 34
Pelanggan dari kelompok 36 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 36 selesai di loket 1 menuju antrian loket 2 pada 2946.23
Jumlah pelanggan di Loket 1: 2
Pelanggan dari Kelompok 36 memasuki antrean loket 2 pada 2946.23
Jumlah pelanggan di Loket 2: 1
Loket Pemesanan menerima pelanggan dari kelompok 37 pada 2946.23
Loket Pemesanan menerima pelanggan dari kelompok 36 pada 2946.23
Pelanggan dari kelompok 37 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 37 selesai di loket 1 menuju antrian loket 2 pada 2961.32
Jumlah pelanggan di Loket 1: 1
Pelanggan dari Kelompok 37 memasuki antrean 

Pelanggan dari Kelompok 49 memasuki antrean loket 1 pada 3758.16
Jumlah pelanggan di Loket 1: 2
Pelanggan dari kelompok 48 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 48 selesai di loket 1 menuju antrian loket 2 pada 3765.37
Jumlah pelanggan di Loket 1: 1
Pelanggan dari Kelompok 48 memasuki antrean loket 2 pada 3765.37
Jumlah pelanggan di Loket 2: 1
Loket Pemesanan menerima pelanggan dari kelompok 49 pada 3765.37
Loket Pemesanan menerima pelanggan dari kelompok 48 pada 3765.37
Total pelanggan sejauh ini: 67
PELANGGAN DATANG!
Group ID: 50
Waktu Kedatangan: 3767.19
Jumlah Pelanggan: 1
      ***         
Pelanggan dari Kelompok 50 memasuki antrean loket 1 pada 3767.19
Jumlah pelanggan di Loket 1: 2
Pelanggan dari kelompok 48 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 48 selesai di loket 2 pada 3775.52
Jumlah pelanggan di Loket 2: 0
Pelanggan dari Kelompok 48 memilih Take-Away
Pelanggan dari kelompok 49 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 49 selesai di l

...

Pelanggan dari Kelompok 90 selesai di loket 2 pada 6889.56
Jumlah pelanggan di Loket 2: 2
Pelanggan dari Kelompok 90 memilih Dine-in
Pelanggan dari Kelompok 90 menempati 1 kursi pada 6889.56
Kursi tersisa : 34
Loket Pemesanan menerima pelanggan dari kelompok 91 pada 6889.56
Pelanggan dari kelompok 93 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 93 selesai di loket 1 menuju antrian loket 2 pada 6904.9
Jumlah pelanggan di Loket 1: 0
Pelanggan dari Kelompok 93 memasuki antrean loket 2 pada 6904.9
Jumlah pelanggan di Loket 2: 3
Total pelanggan sejauh ini: 122
PELANGGAN DATANG!
Group ID: 94
Waktu Kedatangan: 6955.87
Jumlah Pelanggan: 1
      ***         
Pelanggan dari Kelompok 94 memasuki antrean loket 1 pada 6955.87
Jumlah pelanggan di Loket 1: 1
Loket Pemesanan menerima pelanggan dari kelompok 94 pada 6955.87
Pelanggan dari kelompok 91 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 91 selesai di loket 2 pada 6963.68
Jumlah pelanggan di Loket 2: 2
Pelanggan dari Kelompok 91 

Pelanggan dari Kelompok 104 selesai di loket 1 menuju antrian loket 2 pada 8043.5
Jumlah pelanggan di Loket 1: 0
Pelanggan dari Kelompok 104 memasuki antrean loket 2 pada 8043.5
Jumlah pelanggan di Loket 2: 2
Pelanggan dari kelompok 103 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 103 selesai di loket 2 pada 8049.47
Jumlah pelanggan di Loket 2: 1
Pelanggan dari Kelompok 103 memilih Take-Away
Loket Pemesanan menerima pelanggan dari kelompok 104 pada 8049.47
Pelanggan dari kelompok 104 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 104 selesai di loket 2 pada 8065.1
Jumlah pelanggan di Loket 2: 0
Pelanggan dari Kelompok 104 memilih Take-Away
Total pelanggan sejauh ini: 136
PELANGGAN DATANG!
Group ID: 105
Waktu Kedatangan: 8154.71
Jumlah Pelanggan: 2
      ***         
Pelanggan dari Kelompok 105 memasuki antrean loket 1 pada 8154.71
Jumlah pelanggan di Loket 1: 1
Loket Pemesanan menerima pelanggan dari kelompok 105 pada 8154.71
Total pelanggan sejauh ini: 137
PELANGGAN DATA

Pelanggan dari kelompok 114 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 114 selesai di loket 2 pada 10088.41
Jumlah pelanggan di Loket 2: 1
Pelanggan dari Kelompok 114 memilih Dine-in
Pelanggan dari Kelompok 114 menempati 1 kursi pada 10088.41
Kursi tersisa : 34
Loket Pemesanan menerima pelanggan dari kelompok 115 pada 10088.41
Pelanggan dari kelompok 115 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 115 selesai di loket 2 pada 10094.76
Jumlah pelanggan di Loket 2: 0
Pelanggan dari Kelompok 115 memilih Dine-in
Pelanggan dari Kelompok 115 menempati 1 kursi pada 10094.76
Kursi tersisa : 33
Total pelanggan sejauh ini: 148
PELANGGAN DATANG!
Group ID: 116
Waktu Kedatangan: 10162.3
Jumlah Pelanggan: 1
      ***         
Pelanggan dari Kelompok 116 memasuki antrean loket 1 pada 10162.3
Jumlah pelanggan di Loket 1: 1
Loket Pemesanan menerima pelanggan dari kelompok 116 pada 10162.3
Pelanggan dari kelompok 116 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 116 selesai di l

Pelanggan dari Kelompok 126 memilih Take-Away
Total pelanggan sejauh ini: 164
PELANGGAN DATANG!
Group ID: 127
Waktu Kedatangan: 12213.33
Jumlah Pelanggan: 2
      ***         
Pelanggan dari Kelompok 127 memasuki antrean loket 1 pada 12213.33
Jumlah pelanggan di Loket 1: 1
Loket Pemesanan menerima pelanggan dari kelompok 127 pada 12213.33
Pelanggan dari kelompok 127 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 127 selesai di loket 1 menuju antrian loket 2 pada 12231.7
Jumlah pelanggan di Loket 1: 0
Pelanggan dari Kelompok 127 memasuki antrean loket 2 pada 12231.7
Jumlah pelanggan di Loket 2: 1
Loket Pemesanan menerima pelanggan dari kelompok 127 pada 12231.7
Pelanggan dari kelompok 127 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 127 selesai di loket 2 pada 12240.55
Jumlah pelanggan di Loket 2: 0
Pelanggan dari Kelompok 127 memilih Take-Away
Total pelanggan sejauh ini: 165
PELANGGAN DATANG!
Group ID: 128
Waktu Kedatangan: 12357.88
Jumlah Pelanggan: 1
      ***         


Pelanggan dari Kelompok 137 memilih Dine-in
Pelanggan dari Kelompok 137 menempati 3 kursi pada 13571.2
Kursi tersisa : 33
Loket Pemesanan menerima pelanggan dari kelompok 138 pada 13571.2
Pelanggan dari kelompok 138 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 138 selesai di loket 2 pada 13623.09
Jumlah pelanggan di Loket 2: 0
Pelanggan dari Kelompok 138 memilih Take-Away
Pelanggan dari Kelompok 115 selesai Dine-In pada 13675.15
Kursi bertambah. Jumlah kursi yang tersedia : 34
Pelanggan dari Kelompok 115 memutuskan memesan lagi pada 13675.15
Total pelanggan sejauh ini: 181
PELANGGAN DATANG!
Group ID: 139
Waktu Kedatangan: 13848.38
Jumlah Pelanggan: 1
      ***         
Pelanggan dari Kelompok 139 memasuki antrean loket 1 pada 13848.38
Jumlah pelanggan di Loket 1: 1
Loket Pemesanan menerima pelanggan dari kelompok 139 pada 13848.38
Pelanggan dari kelompok 139 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 139 selesai di loket 1 menuju antrian loket 2 pada 13908.77
Jumlah p

In [60]:
#statistik dari hasil simulasinya :

println("Total jumlah pelanggan yang terlibat dalam simulasi: $total_customers")
println("Waktu simulasi : 14400 detik")

#rata-rata panjang antrian persatuan waktu,
waktu_simulasi = 14400
mean_queue_lengths1 = bakery.areaUnderQ1/waktu_simulasi 
mean_queue_lengths2 = bakery.areaUnderQ2/waktu_simulasi
mean_queue_lengths0 = bakery.areaUnderQ0/waktu_simulasi
println("Rata-rata panjang antrian di Loket 1 adalah $(round(mean_queue_lengths1,digits=0))")
println("Rata-rata panjang antrian di Loket 2 adalah $(round(mean_queue_lengths2,digits=0))")
println("Rata-rata panjang antrian di Loket Snackbox adalah $(round(mean_queue_lengths0,digits=1))")

#rata-rata lama waktu tunggu loket 1,
mean_wait_loket1 = round((ct.wait_loket1 |> mean),digits=0)
#rata-rata lama waktu tunggu loket 2,
mean_wait_loket2 = round((ct.wait_loket2 |> mean),digits=0)
#rata-rata lama waktu tunggu loket S,
mean_wait_loket0 = round((ct.wait_loket0 |> mean),digits=0)

#rata2 waktu tunggu di tiap loket
println("Rata-rata waktu tunggu di loket 1 adalah $mean_wait_loket1 detik")
println("Rata-rata waktu tunggu di loket 2 adalah $mean_wait_loket2 detik")
println("Rata-rata waktu tunggu di loket Snackbox adalah $mean_wait_loket0 detik")

#Total pelanggan balk di tiap loket
println("Total pelanggan balk di Loket 1: $(bakery.balk_count1)")
println("Total pelanggan balk di Loket 2: $(bakery.balk_count2)")
println("Total pelanggan balk di Loket Snackbox: $(bakery.balk_count0)")

#Jumlah pelanggan yg gabisa dine in beralih take away
println("Jumlah pelanggan yang ingin Dine-In tetapi kursi penuh: $(bakery.dinein_rejected_count)")

Total jumlah pelanggan yang terlibat dalam simulasi: 187
Waktu simulasi : 14400 detik
Rata-rata panjang antrian di Loket 1 adalah 0.0
Rata-rata panjang antrian di Loket 2 adalah 1.0
Rata-rata panjang antrian di Loket Snackbox adalah 0.0
Rata-rata waktu tunggu di loket 1 adalah 17.0 detik
Rata-rata waktu tunggu di loket 2 adalah 26.0 detik
Rata-rata waktu tunggu di loket Snackbox adalah 0.0 detik
Total pelanggan balk di Loket 1: 0
Total pelanggan balk di Loket 2: 0
Total pelanggan balk di Loket Snackbox: 0
Jumlah pelanggan yang ingin Dine-In tetapi kursi penuh: 0


In [61]:
ct.wait_loket0

8-element Vector{Any}:
  0.0
  0.0
  0.0
 -0.0
 -0.0
  0.0
  0.0
 -0.0

In [62]:
# Fungsi utama simulasi
function run_simulation(seed)
    Random.seed!(seed)
    global total_customers
    total_customers = 0  # Reset total pelanggan


    # Panggil arrival_customer langsung
    env = Simulation()
    cust = CUSTOMER() #inisialisasi ulang CUSTOMER pada setiap iterasi
    ct = CATATAN()
    bakery = BAKERY(env)
    @process arrival_customer(env, bakery)

    # Jalankan simulasi
    run(env, 14400)  # Waktu simulasi 4 jam

    # Statistik
    return Dict(
        :mean_queue_length1 => bakery.areaUnderQ1 / 14400,
        :mean_queue_length2 => bakery.areaUnderQ2 / 14400,
        :mean_queue_length0 => bakery.areaUnderQ0/ 14400,
        :total_customers => total_customers,
        :total_balk1 => bakery.balk_count1,
        :total_balk2 => bakery.balk_count2,
        :total_balk0 => bakery.balk_count0,
        :dinein_rejected_count => bakery.dinein_rejected_count,
        :mean_waitloket1 => mean_wait_loket1,
        :mean_waitloket2 => mean_wait_loket2,
        :mean_waitloket0 => mean_wait_loket0,
    )
end

run_simulation (generic function with 1 method)

In [63]:
# Jalankan simulasi 30 kali
results = []
for i in 1:30
    seed = i * 1234
    push!(results, run_simulation(seed))
end

Total pelanggan sejauh ini: 2
PELANGGAN DATANG!
Group ID: 1
Waktu Kedatangan: 18.29
Jumlah Pelanggan: 2
      ***         
Pelanggan dari Kelompok 1 memasuki antrean loket 1 pada 18.29
Jumlah pelanggan di Loket 1: 1
Loket Pemesanan menerima pelanggan dari kelompok 1 pada 18.29
Pelanggan dari kelompok 1 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 1 selesai di loket 1 menuju antrian loket 2 pada 21.3
Jumlah pelanggan di Loket 1: 0
Pelanggan dari Kelompok 1 memasuki antrean loket 2 pada 21.3
Jumlah pelanggan di Loket 2: 1
Loket Pemesanan menerima pelanggan dari kelompok 1 pada 21.3
Pelanggan dari kelompok 1 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 1 selesai di loket 2 pada 38.37
Jumlah pelanggan di Loket 2: 0
Pelanggan dari Kelompok 1 memilih Take-Away
Total pelanggan sejauh ini: 4
PELANGGAN DATANG!
Group ID: 2
Waktu Kedatangan: 157.57
Jumlah Pelanggan: 2
      ***         
Pelanggan dari Kelompok 2 memasuki antrean loket 1 pada 157.57
Jumlah pelanggan di Loket 1: 1
L

Jumlah pelanggan di Loket Snackbox: 0
Pelanggan dari Kelompok 15 memasuki antrean loket 2 pada 867.19
Jumlah pelanggan di Loket 2: 1
Loket Pemesanan menerima pelanggan dari kelompok 15 pada 867.19
Total pelanggan sejauh ini: 24
PELANGGAN DATANG!
Group ID: 17
Waktu Kedatangan: 885.99
Jumlah Pelanggan: 1
      ***         
Pelanggan dari Kelompok 17 memasuki antrean loket 1 pada 885.99
Jumlah pelanggan di Loket 1: 1
Loket Pemesanan menerima pelanggan dari kelompok 17 pada 885.99
Pelanggan dari kelompok 17 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 17 selesai di loket 1 menuju antrian loket 2 pada 904.42
Jumlah pelanggan di Loket 1: 0
Pelanggan dari Kelompok 17 memasuki antrean loket 2 pada 904.42
Jumlah pelanggan di Loket 2: 2
Total pelanggan sejauh ini: 25
PELANGGAN DATANG!
Group ID: 18
Waktu Kedatangan: 917.16
Jumlah Pelanggan: 1
      ***         
Pelanggan dari Kelompok 18 memasuki antrean loket 1 pada 917.16
Jumlah pelanggan di Loket 1: 1
Loket Pemesanan menerima pelanggan

Pelanggan dari Kelompok 28 selesai di loket 2 pada 2214.21
Jumlah pelanggan di Loket 2: 0
Pelanggan dari Kelompok 28 memilih Take-Away
Total pelanggan sejauh ini: 40
PELANGGAN DATANG!
Group ID: 29
Waktu Kedatangan: 2415.85
Jumlah Pelanggan: 2
      ***         
Pelanggan dari Kelompok 29 memasuki antrean loket 1 pada 2415.85
Jumlah pelanggan di Loket 1: 1
Loket Pemesanan menerima pelanggan dari kelompok 29 pada 2415.85
Total pelanggan sejauh ini: 41
PELANGGAN DATANG!
Group ID: 30
Waktu Kedatangan: 2454.47
Jumlah Pelanggan: 1
      ***         
Pelanggan dari Kelompok 30 memasuki antrean loket 1 pada 2454.47
Jumlah pelanggan di Loket 1: 2
Total pelanggan sejauh ini: 43
PELANGGAN DATANG!
Group ID: 31
Waktu Kedatangan: 2459.02
Jumlah Pelanggan: 2
      ***         
Pelanggan dari Kelompok 31 memasuki antrean loket 1 pada 2459.02
Jumlah pelanggan di Loket 1: 3
Pelanggan dari kelompok 29 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 29 selesai di loket 1 menuju antrian loket 2 pada 2

Pelanggan dari Kelompok 41 selesai di loket 1 menuju antrian loket 2 pada 3395.64
Jumlah pelanggan di Loket 1: 0
Pelanggan dari Kelompok 41 memasuki antrean loket 2 pada 3395.64
Jumlah pelanggan di Loket 2: 2
Pelanggan dari kelompok 40 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 40 selesai di loket 2 pada 3411.12
Jumlah pelanggan di Loket 2: 1
Pelanggan dari Kelompok 40 memilih Dine-in
Pelanggan dari Kelompok 40 menempati 1 kursi pada 3411.12
Kursi tersisa : 38
Loket Pemesanan menerima pelanggan dari kelompok 41 pada 3411.12
Total pelanggan sejauh ini: 54
PELANGGAN DATANG!
Group ID: 42
Waktu Kedatangan: 3471.04
Jumlah Pelanggan: 1
      ***         
Pelanggan dari Kelompok 42 memasuki antrean loket 1 pada 3471.04
Jumlah pelanggan di Loket 1: 1
Loket Pemesanan menerima pelanggan dari kelompok 42 pada 3471.04
Total pelanggan sejauh ini: 55
PELANGGAN DATANG!
Group ID: 43
Waktu Kedatangan: 3532.83
Jumlah Pelanggan: 1
      ***         
Pelanggan dari Kelompok 43 memasuki antrean l

PELANGGAN DATANG!
Group ID: 53
Waktu Kedatangan: 4222.8
Jumlah Pelanggan: 1
      ***         
Pelanggan dari Kelompok 53 memasuki antrean loket 1 pada 4222.8
Jumlah pelanggan di Loket 1: 1
Loket Pemesanan menerima pelanggan dari kelompok 53 pada 4222.8
Pelanggan dari kelompok 52 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 52 selesai di loket 2 pada 4263.84
Jumlah pelanggan di Loket 2: 0
Pelanggan dari Kelompok 52 memilih Take-Away
Pelanggan dari kelompok 53 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 53 selesai di loket 1 menuju antrian loket 2 pada 4295.19
Jumlah pelanggan di Loket 1: 0
Pelanggan dari Kelompok 53 memasuki antrean loket 2 pada 4295.19
Jumlah pelanggan di Loket 2: 1
Loket Pemesanan menerima pelanggan dari kelompok 53 pada 4295.19
Pelanggan dari kelompok 53 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 53 selesai di loket 2 pada 4296.7
Jumlah pelanggan di Loket 2: 0
Pelanggan dari Kelompok 53 memilih Take-Away
Pelanggan dari Kelompok 51 selesai 

...

Pelanggan dari kelompok 86 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 86 selesai di loket 1 menuju antrian loket 2 pada 5662.41
Jumlah pelanggan di Loket 1: 0
Pelanggan dari Kelompok 86 memasuki antrean loket 2 pada 5662.41
Jumlah pelanggan di Loket 2: 4
Pelanggan dari kelompok 83 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 83 selesai di loket 2 pada 5684.62
Jumlah pelanggan di Loket 2: 3
Pelanggan dari Kelompok 83 memilih Take-Away
Loket Pemesanan menerima pelanggan dari kelompok 84 pada 5684.62
Pelanggan dari kelompok 84 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 84 selesai di loket 2 pada 5729.68
Jumlah pelanggan di Loket 2: 2
Pelanggan dari Kelompok 84 memilih Take-Away
Loket Pemesanan menerima pelanggan dari kelompok 85 pada 5729.68
Pelanggan dari kelompok 85 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 85 selesai di loket 2 pada 5749.78
Jumlah pelanggan di Loket 2: 1
Pelanggan dari Kelompok 85 memilih Take-Away
Loket Pemesanan menerima pelangg

Loket Pemesanan menerima pelanggan dari kelompok 98 pada 6805.36
Total pelanggan sejauh ini: 127
PELANGGAN DATANG!
Group ID: 99
Waktu Kedatangan: 6830.09
Jumlah Pelanggan: 1
      ***         
Pelanggan dari Kelompok 99 memasuki antrean loket 1 pada 6830.09
Jumlah pelanggan di Loket 1: 2
Pelanggan dari kelompok 96 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 96 selesai di loket 2 pada 6855.42
Jumlah pelanggan di Loket 2: 1
Pelanggan dari Kelompok 96 memilih Take-Away
Loket Pemesanan menerima pelanggan dari kelompok 97 pada 6855.42
Pelanggan dari kelompok 97 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 97 selesai di loket 2 pada 6874.29
Jumlah pelanggan di Loket 2: 0
Pelanggan dari Kelompok 97 memilih Dine-in
Pelanggan dari Kelompok 97 menempati 1 kursi pada 6874.29
Kursi tersisa : 34
Pelanggan dari kelompok 98 meninggalkan Loket Pemesanan
Pelanggan dari Kelompok 98 selesai di loket 1 menuju antrian loket 2 pada 6877.45
Jumlah pelanggan di Loket 1: 1
Pelanggan dari Kelom

Pelanggan dari Kelompok 107 memilih Take-Away
Loket Snackbox menerima pelanggan dari kelompok 108 pada 7348.01
Pelanggan dari kelompok 110 meninggalkan Loket Snackbox
Pelanggan dari Kelompok 110 selesai di loket Snackbox menuju antrian loket 2 pada 7349.44
Jumlah pelanggan di Loket Snackbox: 0
Pelanggan dari Kelompok 110 memasuki antrean loket 2 pada 7349.44
Jumlah pelanggan di Loket 2: 3
Pelanggan dari kelompok 108 meninggalkan Loket Snackbox
Pelanggan dari Kelompok 108 selesai di loket 2 pada 7369.82
Jumlah pelanggan di Loket 2: 2
Pelanggan dari Kelompok 108 memilih Take-Away
Loket Snackbox menerima pelanggan dari kelompok 109 pada 7369.82
Total pelanggan sejauh ini: 141
PELANGGAN DATANG!
Group ID: 111
Waktu Kedatangan: 7382.21
Jumlah Pelanggan: 1
      ***         
Pelanggan dari Kelompok 111 memasuki antrean loket 1 pada 7382.21
Jumlah pelanggan di Loket 1: 1
Loket Pemesanan menerima pelanggan dari kelompok 111 pada 7382.21
Pelanggan dari kelompok 111 meninggalkan Loket Pemesanan
P

Excessive output truncated after 524339 bytes.


Pelanggan dari kelompok 120 meninggalkan Loket Pembayaran
Pelanggan dari Kelompok 120 selesai di loket 2 pada 8568.79
Jumlah pelanggan di Loket 2: 0
Pelanggan dari Kelompok 120 memilih Take-Away
Total pelanggan sejauh ini: 155
PELANGGAN DATANG!
Group ID: 122
Waktu Kedatangan: 8610.16
Jumlah Pelanggan: 1
      ***         
Pelanggan dari Kelompok 122 memasuki antrean loket 1 pada 8610.16
Jumlah pelanggan di Loket 1: 1
Loket Pemesanan menerima pelanggan dari kelompok 122 pada 8610.16
Total pelanggan sejauh ini: 156
PELANGGAN DATANG!
Group ID: 123
Waktu Kedatangan: 8626.59
Jumlah Pelanggan: 1
      ***         
Pelanggan dari Kelompok 123 memasuki antrean loket 1 pada 8626.59
Jumlah pelanggan di Loket 1: 2
Total pelanggan sejauh ini: 157
PELANGGAN DATANG!
Group ID: 124
Waktu Kedatangan: 8686.5
Jumlah Pelanggan: 1
      ***         
Pelanggan dari Kelompok 124 memasuki antrean loket 1 pada 8686.5
Jumlah pelanggan di Loket 1: 3
Total pelanggan sejauh ini: 158
PELANGGAN DATANG!
Group ID: 12

In [64]:
# Statistik hasil
mean_queue_lengths1_all = mean([r[:mean_queue_length1] for r in results])
mean_queue_lengths2_all = mean([r[:mean_queue_length2] for r in results])
mean_queue_lengths0_all = mean([r[:mean_queue_length0] for r in results])
total_balk1_all = mean([r[:total_balk1] for r in results])
total_balk2_all = mean([r[:total_balk2] for r in results])
total_balk0_all = mean([r[:total_balk0] for r in results])
dinein_rejected_count_all = mean([r[:dinein_rejected_count] for r in results])
mean_total_customers_all = mean([r[:total_customers] for r in results])
mean_wait_loket1_all = mean([r[:mean_waitloket1] for r in results])
mean_wait_loket2_all = mean([r[:mean_waitloket2] for r in results])
mean_wait_loket0_all = mean([r[:mean_waitloket0] for r in results])

# Menampilkan hasil
println("Rata-rata panjang antrean di Loket 1: $(round(mean_queue_lengths1_all, digits=0))")
println("Rata-rata panjang antrean di Loket 2: $(round(mean_queue_lengths2_all, digits=0))")
println("Rata-rata panjang antrean di Loket Snackbox: $(round(mean_queue_lengths0_all, digits=1))")
println("Total pelanggan balk di Loket 1: $(round(total_balk1_all, digits=0))")
println("Total pelanggan balk di Loket 2: $(round(total_balk2_all, digits=0))")
println("Total pelanggan balk di Loket Snackbox: $(round(total_balk0_all, digits=0))")
println("Jumlah pelanggan gagal Dine-In: $(round(dinein_rejected_count_all, digits=0))")
println("Rata-rata pelanggan yang datang: $(round(mean_total_customers_all, digits=0))")
println("Rata-rata waktu tunggu pelanggan di loket pemesanan :$(round(mean_wait_loket1_all, digits=0))")
println("Rata-rata waktu tunggu pelanggan di loket pembayaran :$(round(mean_wait_loket2_all, digits=0))")
println("Rata-rata waktu tunggu pelanggan di loket snackbox :$(round(mean_wait_loket0_all, digits=0))")

Rata-rata panjang antrean di Loket 1: 1.0
Rata-rata panjang antrean di Loket 2: 1.0
Rata-rata panjang antrean di Loket Snackbox: 0.0
Total pelanggan balk di Loket 1: 0.0
Total pelanggan balk di Loket 2: 0.0
Total pelanggan balk di Loket Snackbox: 0.0
Jumlah pelanggan gagal Dine-In: 0.0
Rata-rata pelanggan yang datang: 198.0
Rata-rata waktu tunggu pelanggan di loket pemesanan :17.0
Rata-rata waktu tunggu pelanggan di loket pembayaran :26.0
Rata-rata waktu tunggu pelanggan di loket snackbox :0.0
